# Использование модели метчинга для поиска топ-5 услуг для новых услуг клиник

In [1]:
import pandas as pd
import numpy as np
from matcher.matcher import SentenceSimilarity
from configs.config_matcher import MatcherConfig
from utils.compute_metrics import mapk, mark
from tqdm import tqdm
tqdm.pandas()

/home/evgen/desktop/ranker/.venv/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
2024-09-18 22:49:17.678975: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-18 22:49:17.687710: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-18 22:49:17.696795: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already 

In [2]:
data = pd.read_csv("data/data_for_matcher.csv")

Для проверки работы алгоритма, будет использоваться датасет из которого удалены дубликаты в столбце `local_name`

Параметры для конфига указаны в файле `configs/config_matcher.py`

In [3]:
config = MatcherConfig()
model = SentenceSimilarity(config=config)

In [4]:
predict = data['service_name'].progress_apply(lambda x: model.get_top_k(x, top_k=5))

  0%|          | 0/4047 [00:00<?, ?it/s]/home/evgen/desktop/ranker/matcher/matcher.py:60: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(self.embedder.encode(sentences=texts, convert_to_tensor=True)).clone().detach()
100%|██████████| 4047/4047 [00:30<00:00, 131.84it/s]


In [5]:
predict_list = predict.apply(lambda x: [key['local_name'] for key in x]).values.tolist()
predict_list[:2]

[['Выявление стресса и его стадии',
  'Комплексная оценка оксидативного стресса (7 параметров)',
  'Комплексное исследование, ОСМ',
  'Обследование домашнего персонала (расширенное)',
  'Комплексное исследование на гормоны (12 показателей)'],
 ['Клинический анализ крови: общий анализ, лейкоцитарная формула, СОЭ (с обязательной микроскопией мазка крови)',
  'Клинический анализ крови: общий анализ, лейкоцитарная формула, СОЭ (с микроскопией мазка крови при выявлении патологических изменений)',
  'Лейкоцитарная формула (с обязательной микроскопией мазка крови)',
  'Лейкоцитарная формула (с микроскопией мазка крови при выявлении патологических изменений)',
  'Клинический анализ крови с микроскопией лейкоцитарной формулы']]

In [6]:
actual_list = data['local_name'].apply(lambda x: [x]).values.tolist()
actual_list[:5]

[['Выявление стресса и его стадии'],
 ['Клинический анализ крови с микроскопией лейкоцитарной формулы'],
 ['Лейкоцитарная формула (с обязательной микроскопией мазка крови)'],
 ['Клинический анализ крови (c лейкоцитарной формулой)'],
 ['Клинический анализ крови: общий анализ, лейкоцитарная формула, СОЭ (с микроскопией мазка крови при выявлении патологических изменений)']]

In [7]:
mark(data['local_name'].apply(lambda x: [x]).values.tolist(), predict_list, k=5)

0.9567580924141339

In [8]:
mapk(data['local_name'].apply(lambda x: [x]).values.tolist(), predict_list, k=5)

0.8989580759410262

посчитать метрики с разными K и сделать сводная таблица с метриками.
По строкам идёт индекс первой строк mapk, mark, f1, rmm, ngdc, а по столбцам идут K, а в пересечении метрика